In [1]:
from ipynb.fs.full.common import *

In [115]:
ACTIVE_SHAM = 'Sham'
GROUP1 = 'PRE-AVG'
GROUP2= 'POST-AVG'
COMPUTE_DATA = True
mapping = {0: 'theta', 1: 'alpha', 2:'beta'}
FREQ_BANDS = {
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
}
# PATIENTS = ['Hemlata', 'Malti', 'Preeti', 'Sharifa', 'Vinita'] if ACTIVE_SHAM == 'Active' else [] # ['Geeta', 'Jitendra', 'Jyoti', 'Kuldeep', 'Seema', 'VijayLaxmi']

# COMPUTE DATA

In [116]:
if COMPUTE_DATA:
    if GROUP1 in ['PRE-AVG'] and GROUP2 in ['POST-AVG']:
        folder1 = '2Pre-TDCS'
        folder2 = '4Post-TDCS'

PLI computations

In [117]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


Group 1 data

In [118]:
if COMPUTE_DATA:
    c = 0
    PSD_1 = []
    con_mat_1 = {'theta': [], 'alpha': [], 'beta': []}
    for ELECTRODES in ['32electrodes', '32electrodes-old']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder1)
        for patient in os.listdir(folder_path):
            print(patient)
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_1 = data_transformation_easy(_)
                raw_1 = g1_preprocess(raw_1)
                # fig = mne.viz.plot_raw_psd(raw_1, picks=raw_1.info['ch_names'], average=True, fmax=FMAX, show=False, color='red')
                psd_1, frequencies = mne.time_frequency.psd_array_welch(raw_1.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ, n_fft = 8192)
                PSD_1.append(psd_1)

                # Epoching
                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    # Manually created events
                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_1):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=(0,0), detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_1.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_1, i)
                    con_mat_1[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_1)

Kuldeep
Creating RawArray with float64 data, n_channels=32, n_times=74999
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:252: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


SSP projectors applied...
Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivit

...

SSP projectors applied...
Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivit

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:252: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Effective window size : 16.384 (s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    comp

Group 2 data

In [119]:
if COMPUTE_DATA:
    c = 0
    con_mat_2 = {'theta': [], 'alpha': [], 'beta': []}
    PSD_2 = []
    for ELECTRODES in ['32electrodes', '32electrodes-old']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder2)
        for patient in os.listdir(folder_path):
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_2 = data_transformation_easy(_)
                raw_2 = g1_preprocess(raw_2)
                # fig = mne.viz.plot_raw_psd(raw_2, picks=raw_2.info['ch_names'], average=True, fmax=FMAX, show=False, color='red')
                psd_2, frequencies = mne.time_frequency.psd_array_welch(raw_2.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ, n_fft = 8192)
                PSD_2.append(psd_2)

                # Epoching
                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    # Manually created events
                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_2):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=(0,0), detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_2.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_2, i)
                    con_mat_2[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_2)

Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:252: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Effective window size : 16.384 (s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    comp

...

Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    comp

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:252: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Effective window size : 16.384 (s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    comp

1. PSD Plotting

PSD_x dimensions = no. of active/sham X n_channels X 23

In [120]:
fig, ax = plt.subplots()
labels = np.arange(0, FMAX)
avg_abs_power_1, avg_abs_power_2 = np.mean(np.mean(PSD_1, axis=0), axis=0), np.mean(np.mean(PSD_2, axis=0), axis=0) 

plt.figure()
plt.plot(frequencies, avg_abs_power_1, color='blue', label=f'{GROUP1}')
plt.plot(frequencies, avg_abs_power_2, color='red', label=f'{GROUP2}')
plt.title(f'Average Absolute Power in {ACTIVE_SHAM} treatment')
plt.xlabel('Frequency (Hz)')
plt.xticks(labels, labels, rotation ='vertical') 
plt.ylabel('PSD (μV²/Hz)')
plt.ylim([0, 2e-11])
plt.legend()

vertical_lines = list(set([limit for band in FREQ_BANDS.values() for limit in band]))
for line in vertical_lines:
    plt.axvline(x=line, color='gray', linestyle='--')
plt.show()

In [121]:
# Average con_mat_1 and con_mat_2
# WE HAVE BOTH INDIVIDUAL FUNCNINAL CONNECIVTY MATRICES OF LL 12 SAMPLES AND AVERAG OF ALL THOSE FOR BOTH GROUPS
for i in mapping:
    connectivity[f'{GROUP1}-{mapping[i]}'] = np.mean(con_mat_1[mapping[i]], axis=0)
    connectivity[f'{GROUP2}-{mapping[i]}'] = np.mean(con_mat_2[mapping[i]], axis=0)

### Statistical analysis - Non parametrcal permutation testing

In [122]:
def create_random_graph(n, e):
    adj_matrix = np.zeros((n, n))
    edges = [(i, j) for i in range(n) for j in range(i) if i != j]
    selected_edges = np.random.choice(len(edges), e, replace=False)
    for edge_idx in selected_edges:
        i, j = edges[edge_idx]
        adj_matrix[i, j] = 1
    return adj_matrix

In [123]:
def make_symmetric(arr, n_channels):
    for row in range(n_channels):
        for col in range(n_channels):
            arr[row][col] = arr[col][row]
    return arr

In [124]:
def calculate_swn(mat, threshold):
    tthresh = np.median(mat[mat!=0]) + threshold*np.std(mat[mat!=0])
    print("*****111111******", tthresh)
    binmat= mat > tthresh
    n_connections = np.count_nonzero(binmat == 1)
    print("*****222222******", n_connections)
    binmat = make_symmetric(binmat.astype(int), n_channels)
    g= calculate_avergae_components(nx.from_numpy_array(binmat))
    Lw_binarized, CC_binarized = g[0], g[3]
    return (CC_binarized/Lw_binarized if n_connections else 0), n_connections

In [125]:
WINDOW_SIZE = 5
def moving_average(data, window_size = WINDOW_SIZE):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [126]:
import matplotlib.pyplot as plt
import random

n_perms = 1000
n_network = n_perms//10
thresholds = np.linspace(0, 1.5, 200)
swn_z_1, swn_z_2 = {}, {}
optimal_threshold = {}
optimal_swn_perms, optimal_swn_real1, optimal_swn_real2 = {}, {}, {}
moving_avg1, moving_avg2 = {}, {}

for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    swn_z_1[mapping[i]], swn_z_2[mapping[i]] = np.zeros(len(thresholds)), np.zeros(len(thresholds))
    optimal_swn_perms[mapping[i]], optimal_swn_real1[mapping[i]], optimal_swn_real2[mapping[i]] = [[] for _ in range(len(thresholds))], np.zeros(len(thresholds)),  np.zeros(len(thresholds))
    max_diff = 0
    for threshi, threshold in enumerate(thresholds):
        print("00000000", threshi, threshold)
        swn_1, n_connections1 = calculate_swn(conmat1, threshold)
        swn_2, n_connections2 = calculate_swn(conmat2, threshold)

        # Create #n_network random matrices with vertices: n_channels and edges: average of edges of thresholded binary matrix of both groups 
        # And make a list of #n_network lists of clustering coeffs and shortest avg path length
        Crand, Lrand = [], []
        swn_perms = np.zeros(n_perms)
        for perm in range(n_network):
            random_binmat = create_random_graph(n_channels, (n_connections1+n_connections2)//2)
            random_binmat = make_symmetric(random_binmat, n_channels)
            grand = calculate_avergae_components(nx.from_numpy_array(random_binmat))
            Crand.append(grand[3])
            Lrand.append(grand[0])

        print("CHECK", Crand.count(0))
        print("CHECK2", Lrand.count(0))
        # choose any 2 random network from above n_networks #n_perms times 
        for perm in range(n_perms):
            whichnetworks2use = random.sample(range(n_network), 2)
            if Crand[whichnetworks2use[1]] and Lrand[whichnetworks2use[0]]:
                swn_perms[perm] = (Crand[whichnetworks2use[0]] / Crand[whichnetworks2use[1]]) / (Lrand[whichnetworks2use[0]] / Lrand[whichnetworks2use[1]])
            else:
                perm -= 1
    
        swn_rand_avg = np.mean(Crand, axis=0)/np.mean(Lrand, axis=0)
        swn_real_1 = swn_1 / swn_rand_avg # TODO: different random avg or same ?
        swn_real_2 = swn_2 / swn_rand_avg
        val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)
        swn_z_1[mapping[i]][threshi] = val1
        swn_z_2[mapping[i]][threshi] = val2


        max_diff = 0
        moving_avg1[mapping[i]] = moving_average(swn_z_1[mapping[i]])
        moving_avg2[mapping[i]] = moving_average(swn_z_2[mapping[i]])
        moving_threshold = moving_average(thresholds)

        # print("$$$$$", optimal_swn_perms[mapping[i]])
        optimal_swn_perms[mapping[i]][threshi] = swn_perms
        optimal_swn_real1[mapping[i]][threshi] =swn_real_1
        optimal_swn_real2[mapping[i]][threshi] =swn_real_2

00000000 0 0.0
*****111111****** 0.38162878787878785
*****222222****** 248
*****111111****** 0.46070075757575757
*****222222****** 247
CHECK 0
CHECK2 0
00000000 1 0.007537688442211055
*****111111****** 0.3823274943836474
*****222222****** 247
*****111111****** 0.461569279753924
*****222222****** 246
CHECK 0
CHECK2 0
00000000 2 0.01507537688442211
*****111111****** 0.38302620088850686
*****222222****** 245
*****111111****** 0.46243780193209033
*****222222****** 245
CHECK 0
CHECK2 0
00000000 3 0.022613065326633167
*****111111****** 0.3837249073933664
*****222222****** 244
*****111111****** 0.46330632411025674
*****222222****** 244
CHECK 0
CHECK2 0
00000000 4 0.03015075376884422
*****111111****** 0.3844236138982259
*****222222****** 244
*****111111****** 0.46417484628842315
*****222222****** 242
CHECK 0
CHECK2 0
00000000 5 0.03768844221105527
*****111111****** 0.3851223204030854
*****222222****** 244
*****111111****** 0.46504336846658956
*****222222****** 239
CHECK 0
CHECK2 0
00000000 6 0

In [ ]:
# index = {}
# for i in mapping:
#     max_diff = 0
#     for mov in range(len(moving_threshold)):
#         x = np.abs(moving_avg1[mapping[i]][mov] - moving_avg2[mapping[i]][mov])
#         if x>max_diff:
#             max_diff = x
#             optimal_threshold[mapping[i]] = moving_threshold[mov]
#             index[mapping[i]] = mov

#     # TODO: save fig file
#     p_value_1 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real1[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
#     p_value_2 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real2[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
#     print(mapping[i], "p-value: ", p_value_1, p_value_2)

#     plt.hist(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)], bins=50, color='b', alpha=0.7)
#     plt.axvline(optimal_swn_real1[mapping[i]][mov+(WINDOW_SIZE//2)], color='m', linewidth=3, label=f'{GROUP1}')
#     plt.axvline(optimal_swn_real2[mapping[i]][mov+(WINDOW_SIZE//2)], color='r', linewidth=3, label=f'{GROUP2}')
#     plt.xlabel(f'{mapping[i]}')
#     plt.xlim([0,10])
#     plt.legend()
#     plt.title('Random Network distribution for both Pre and Post groups')
#     plt.show()

In [ ]:
# To maximize the statistical significance of small world networkness for both groups while 
# maximizing the difference between the z-normalized small world networks of the two groups.
f, axarr = plt.subplots(1,3, figsize=(100, 5))
index = {}
for count, i in enumerate(mapping):
    max_diff = 0
    for mov in range(len(moving_threshold)):
        x = np.abs(moving_avg1[mapping[i]][mov] - moving_avg2[mapping[i]][mov])
        if x>max_diff:
            max_diff = x
            optimal_threshold[mapping[i]] = moving_threshold[mov]
            index[mapping[i]] = mov

    # TODO: save fig file
    p_value_1 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real1[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
    p_value_2 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real2[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
    print(mapping[i], "p-value: ", p_value_1, p_value_2)

    axarr[count].hist(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)], bins=50, color='b', alpha=0.7)
    axarr[count].axvline(optimal_swn_real1[mapping[i]][mov+(WINDOW_SIZE//2)], color='m', linewidth=3, label=f'{GROUP1}')
    axarr[count].axvline(optimal_swn_real2[mapping[i]][mov+(WINDOW_SIZE//2)], color='r', linewidth=3, label=f'{GROUP2}')
    axarr[count].set_xlabel(mapping[i])
    axarr[count].set_xlim([0,12])
    # axarr[count].set_ylim([0,100])
    axarr[count].legend()
f.suptitle('Random Network distribution for both Pre and Post groups')
plt.show()

theta p-value:  0.0 0.0
alpha p-value:  0.0 0.0
beta p-value:  0.014 0.0


In [100]:
# f, axarr = plt.subplots(1,3, figsize=(100, 5))
for count, i in enumerate(mapping):
    # High standardized network metric suggests that our network exhibits properties away from random networks 
    plt.plot(thresholds, swn_z_1[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP1}')
    plt.plot(thresholds, swn_z_2[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP2}')
    plt.xlabel(f'Threshold (Standard deviations above median) at frequency band at {mapping[i]}')
    plt.ylabel(f'SWN_z')
    plt.xlim([0, 1.5])
    plt.legend()
    vline = ((index[mapping[i]]+WINDOW_SIZE//2)*1.5)/200
    plt.axvline(x=vline, color='gray', linestyle='--')
    # f.suptitle('Threshold (Standard deviations above median)')
    plt.show()

In [101]:
# Find optimal threhsold
binarized_matrix = {}
print(optimal_threshold)
for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    tthresh1 = np.median(conmat1[conmat1!=0]) + optimal_threshold[mapping[i]]*np.std(conmat1[conmat1!=0])
    tthresh2 = np.median(conmat2[conmat2!=0]) + optimal_threshold[mapping[i]]*np.std(conmat2[conmat2!=0])
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > tthresh1
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > tthresh2
    print(tthresh1, tthresh2)
print(binarized_matrix.keys())


{'theta': 0.1733668341708543, 'alpha': 0.4447236180904522, 'beta': 1.2286432160803997}
0.3383970085483856 0.3832043364955732
0.42906746049382377 0.45147972751117155
0.46942649678715137 0.48795408705981846
dict_keys(['PRE-AVG-theta', 'POST-AVG-theta', 'PRE-AVG-alpha', 'POST-AVG-alpha', 'PRE-AVG-beta', 'POST-AVG-beta'])


In [17]:
# for j in mapping:
#     sw1, sw2 = [], []
#     for k in range(len(con_mat_1[mapping[j]])):
#         arr1 = [x for x in list(con_mat_1[mapping[j]][k][np.tril_indices(32)]) if x!=0]
#         tthresh1 = (1 + np.median(arr1))/np.std(arr1) # wrong threshold TODO: 
#         print(len(arr1), arr1 > tthresh1, "*************")
#         g1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[j]][k] > tthresh1))
#         Lw_1_binarized, CC_1_binarized = g1[0], g1[3]
#         sw1.append(CC_1_binarized/Lw_1_binarized)

#         arr2 = [x for x in list(con_mat_2[mapping[j]][k][np.tril_indices(32)]) if x!=0]
#         tthresh2 = (1 + np.median(arr2))/np.std(arr2)
#         g2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[j]][k] > tthresh2))
#         Lw_2_binarized, CC_2_binarized = g2[0], g2[3]
#         sw2.append(CC_2_binarized/Lw_2_binarized)

#     print(sw1, len(sw1))
#     observed_diff = np.mean(sw2) - np.mean(sw1)
#     combined_data = np.concatenate((sw1, sw2))
#     print(combined_data, len(combined_data))
        
#     permuted_diffs = []
#     for i in range(1000):
#         permuted_data = np.random.permutation(combined_data)
        
#         permuted_pre_data = permuted_data[:len(data1)]
#         permuted_post_data = permuted_data[len(data1):]
        
#         permuted_diff = np.mean(permuted_post_data, axis=0) - np.mean(permuted_pre_data, axis=0)
#         permuted_diffs.append(permuted_diff)

#     print("&&&&&&", permuted_diffs)
#     print("*****", observed_diff)
#     # Calculate p-value
#     # z_value = (observed_diff - np.mean(permuted_diffs))  / np.std(permuted_diffs)

#     print("Observed Difference:", observed_diff)
#     # print("Z-value:", z_value)

#     # Plot the distribution of permuted differences
#     plt.hist(permuted_diffs, bins=30, edgecolor='black')
#     plt.axvline(observed_diff, color='red', linestyle='dashed', linewidth=1, label='Observed Difference')
#     plt.xlabel('Difference in Means')
#     plt.ylabel('Frequency')
#     plt.title(f'Distribution of Null Hypothesis for - {mapping[j]} - band')
#     plt.legend()
#     plt.show()

Making individual matrix symmetric ?

In [18]:
# for i in mapping:
#     for s in range(len(con_mat_1[mapping[i]])):
#         for row in range(n_channels):
#             for col in range(n_channels):
#                 con_mat_1[mapping[i]][s][row][col] = con_mat_1[mapping[i]][s][col][row]
#                 con_mat_2[mapping[i]][s][row][col] = con_mat_2[mapping[i]][s][col][row]

In [19]:
# optimal_threshold = {}
# swn1 = {}
# swn2 = {}
# edges_1 = {}
# edges_2 = {}
# # Generating 50 random networks with same number of vertices and edges 
# random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
# thresholds = np.arange(0.1, 0.9, 0.002)
# table = []

# for i in mapping:
#     swn1[mapping[i]] = []
#     swn2[mapping[i]] = []
#     for s in range(len(con_mat_1[mapping[i]])):
#         edges_1[mapping[i]] = []
#         edges_2[mapping[i]] = []
#         ratios = []
#         for threshold in thresholds:
#             # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
#             g1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]][s] > threshold))
#             Lw_1_binarized, CC_1_binarized = g1[0], g1[3]

#             g2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]][s] > threshold))
#             Lw_2_binarized, CC_2_binarized = g2[0], g2[3]

#             swi_ratio_binarized =  (CC_1_binarized*Lw_2_binarized)/(Lw_1_binarized*CC_2_binarized) if Lw_1_binarized and CC_2_binarized else 0
#             ratios.append(swi_ratio_binarized)
            
#         optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))]
#         # TODO: multiple comparison

#         Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
#         Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

#         graph1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]][s] > optimal_threshold[mapping[i]]))
#         Lw_1_binarized, CC_1_binarized = graph1[0], graph1[3]

#         graph2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]][s] > optimal_threshold[mapping[i]]))
#         Lw_2_binarized, CC_2_binarized = graph2[0], graph2[3]

#         swi_binarized_1 =  (CC_1_binarized/Crand)/(Lw_1_binarized/Lrand)
#         swi_binarized_2 = (CC_2_binarized/Crand)/(Lw_2_binarized/Lrand)
#         print(swn1, "&&&&&&&&&&&&&&&&")
#         swn1[mapping[i]].append(swi_binarized_1)
#         swn2[mapping[i]].append(swi_binarized_2)
#         print(swi_binarized_1, "Cc1 ", CC_1_binarized, "Lw1 ", Lw_1_binarized)
#         print(swi_binarized_2, "Cc2 ", CC_2_binarized, "Lw2 ", Lw_2_binarized)  

#         edges_1[mapping[i]].append(nx.from_numpy_array(con_mat_1[mapping[i]][s] > optimal_threshold[mapping[i]]).number_of_edges())
#         edges_2[mapping[i]].append(nx.from_numpy_array(con_mat_2[mapping[i]][s] > optimal_threshold[mapping[i]]).number_of_edges())

#         print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]}")
        
# print(swn1, swn2)

In [20]:
# for j in mapping:
#     data1 = swn1[mapping[j]]
#     data2 = swn2[mapping[j]]
#     combined_data = np.concatenate((data1, data2))
#     observed_diff = np.mean(data2) - np.mean(data1)

#     permuted_diffs = np.zeros(1000)
#     for i in range(1000):
#         permuted_data = np.random.permutation(combined_data)
        
#         permuted_pre_data = permuted_data[:len(data1)]
#         permuted_post_data = permuted_data[len(data1):]
        
#         permuted_diff = np.mean(permuted_post_data) - np.mean(permuted_pre_data)
#         permuted_diffs[i] = permuted_diff

#     z_value = (observed_diff - np.mean(permuted_diffs))  / np.std(permuted_diffs)
#     print("Observed Difference:", observed_diff)
#     print("Z-value:", z_value)

#     # Plot the distribution of permuted differences
#     plt.hist(permuted_diffs, bins=30, edgecolor='black')
#     plt.axvline(observed_diff, color='red', linestyle='dashed', linewidth=1, label='Observed Difference')
#     plt.xlabel('Difference in Means')
#     plt.ylabel('Frequency')
#     plt.title(f'Distribution of Null Hypothesis for - {mapping[j]} - band')
#     plt.legend()
#     plt.show()

Average both con_mat_1 and con_mat_2 and Make matrix symmetric

In [21]:
# for i in mapping:
#     con_mat_1[mapping[i]] = np.mean(con_mat_1[mapping[i]], axis=0)
#     con_mat_2[mapping[i]] = np.mean(con_mat_2[mapping[i]], axis=0)

# print(len(con_mat_1), con_mat_1, len(con_mat_1['theta']))
# print(len(con_mat_2), con_mat_2, len(con_mat_2['theta']))

### 3. Thresholding - M1 -  Small World Index

In [22]:
# Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
# Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [23]:
# optimal_threshold = {}
# edges_1 = {}
# edges_2 = {}
# # Generating 50 random networks with same number of vertices and edges 
# random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
# thresholds = np.arange(0.1, 0.9, 0.002)
# table = []

# for i in mapping:
#     edges_1[mapping[i]] = []
#     edges_2[mapping[i]] = []
#     ratios = []
#     for threshold in thresholds:
#         # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
#         g1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]] > threshold))
#         Lw_1_binarized, CC_1_binarized = g1[0], g1[3]

#         g2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]] > threshold))
#         Lw_2_binarized, CC_2_binarized = g2[0], g2[3]

#         swi_ratio_binarized =  (CC_1_binarized*Lw_2_binarized)/(Lw_1_binarized*CC_2_binarized) if Lw_1_binarized and CC_2_binarized else 0
#         ratios.append(swi_ratio_binarized)
        
#     optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))]
#     # TODO: multiple comparison

#     # if p_value < 0.01: # Reject the null hypotheses
#     #     Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
#     #     Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

#     #     graph1 = calculate_avergae_components(nx.from_numpy_array(con_mat_1[mapping[i]] > optimal_threshold[mapping[i]]))
#     #     Lw_1_binarized, CC_1_binarized = graph1[0], graph1[3]

#     #     graph2 = calculate_avergae_components(nx.from_numpy_array(con_mat_2[mapping[i]] > optimal_threshold[mapping[i]]))
#     #     Lw_2_binarized, CC_2_binarized = graph2[0], graph2[3]

#     #     swi_binarized_1 =  (CC_1_binarized/Crand)/(Lw_1_binarized/Lrand)
#     #     swi_binarized_2 = (CC_2_binarized/Crand)/(Lw_2_binarized/Lrand)
#     #     print(swi_binarized_1, "Cc1 ", CC_1_binarized, "Lw1 ", Lw_1_binarized)
#     #     print(swi_binarized_2, "Cc2 ", CC_2_binarized, "Lw2 ", Lw_2_binarized)  

#     #     edges_1[mapping[i]].append(nx.from_numpy_array(con_mat_1[mapping[i]] > optimal_threshold[mapping[i]]).number_of_edges())
#     #     edges_2[mapping[i]].append(nx.from_numpy_array(con_mat_2[mapping[i]] > optimal_threshold[mapping[i]]).number_of_edges())

#     #     print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]} (Significant)", p_value)
#     #     table.append([mapping[i], str(swi_binarized_1)+' ('+str(CC_1_binarized)+','+str(Lw_1_binarized)+') ',  str(swi_binarized_2)+' ('+str(CC_2_binarized)+','+str(Lw_2_binarized)+') '])
#     # else:
#     #     table.append([mapping[i], None, None])

In [24]:
# df = pd.DataFrame(table, columns=['FREQ BAND', f'{GROUP1}-CC/Lw', f'{GROUP2}-CC/Lw'])
# df.to_excel('comparison-results/average-swi.xlsx')

Find number of links in each graph

In [102]:
l1 = {}
l2 = {}
for i in mapping:
    links_g1 = 0
    links_g2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_g1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_g2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_g1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_g2)

    l1[mapping[i]] = links_g1
    l2[mapping[i]] = links_g2

PRE-AVG-theta-number of links: K= 221
POST-AVG-theta-number of links: K= 217
PRE-AVG-alpha-number of links: K= 166
POST-AVG-alpha-number of links: K= 158
PRE-AVG-beta-number of links: K= 61
POST-AVG-beta-number of links: K= 70


### 5. Asymmetry in  each band

In [81]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

a1 = {}
a2 = {}
for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    a1[mapping[i]] = lr/rr
    a2[mapping[i]] = lrc/rrc


PRE-AVG:  theta left region:  160 right region:  137 proportion:  1.167883211678832
POST-AVG:  theta left region:  156 right region:  130 proportion:  1.2
PRE-AVG:  alpha left region:  117 right region:  103 proportion:  1.1359223300970873
POST-AVG:  alpha left region:  119 right region:  91 proportion:  1.3076923076923077
PRE-AVG:  beta left region:  47 right region:  34 proportion:  1.3823529411764706
POST-AVG:  beta left region:  59 right region:  37 proportion:  1.5945945945945945


### 6. Difference matrix

In [103]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [104]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [105]:
# ALL_CHANNELS = np.array(['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2',
#        'Fp1', 'AF3', 'Fz', 'FC2', 'Cz', 'CP2', 'PO3', 'O1', 'Oz', 'O2',
#        'PO4', 'Pz', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5',
#        'T7', 'P7'])
# def set_montage(raw):
#     mont1020 = mne.channels.make_standard_montage('standard_1020')
#     ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in ALL_CHANNELS]
#     mont1020_new = mont1020.copy()
#     mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
#     kept_channel_info = [mont1020.dig[x+3] for x in ind]
#     mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
#     print("sjkghdkgu", mont1020_new)
#     return raw.set_montage(mont1020_new)

In [108]:
f, axarr = plt.subplots(1,3) 
for count, i in enumerate(mapping):
    conmat1 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)
    # conmat2 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)

    im = axarr[count].imshow(conmat1)
    axarr[count].set_title(f'Functional connectivity Post {mapping[i]} in {ACTIVE_SHAM} treatment')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')

plt.show()

In [31]:
# difference_matrix = {}
# for count, i in enumerate(mapping):
#     difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
#     # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) 
#     for k in range(n_channels):
#         for l in range(n_channels):
#             if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
#                 difference_matrix[mapping[i]][k][l] = 0

#     difference_matrix[mapping[i]] = make_symmetric(difference_matrix[mapping[i]], n_channels)

#     # # Yellow - Higher synchronisation
#     diff_mat = difference_matrix[mapping[i]]
#     plt.imshow(diff_mat, interpolation='none')
#     plt.title(f'Binary FC difference in {ACTIVE_SHAM} treatment in {mapping[i]}')
#     plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
#     plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
#     plt.grid(True, which='both', linestyle='--', linewidth=0.5)
#     plt.show()
   
#     # plot_sensors_connectivity(
#     #     raw_1.info,
#     #     difference_matrix[mapping[i]],
#     #     cbar_label=f'{mapping[i]}-Connectivity',
#     #     )

In [109]:
difference_matrix = {}
# f, axarr = plt.subplots(1,3) 
from mne.channels import find_layout

f, axarr = plt.subplots(1, 3, figsize=(10,3))
for count, i in enumerate(mapping):
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) 
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    difference_matrix[mapping[i]] = make_symmetric(difference_matrix[mapping[i]], n_channels)

    # # Yellow - Higher synchronisation
    diff_mat = difference_matrix[mapping[i]]
    axarr[count].imshow(diff_mat, interpolation='none')
    axarr[count].set_title(mapping[i])
    axarr[count].set_xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

   
    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

f.suptitle(f'Binary FC difference in {ACTIVE_SHAM} treatment')
plt.show()



In [110]:
def calculate_hub(mat):
        hub =[]
        for ch, ch_name in enumerate(raw_1.info['ch_names']):
            hub.append(np.count_nonzero(mat[ch] == 1))
        return hub
        # layout = find_layout(raw_1.info)
        # plot_topomap(hub, layout.pos, names=raw_1.info['ch_names'], sphere='eeglab')
        # plt.show()

In [111]:
edges_threshold = {}
for i in mapping:
    hub = calculate_hub(difference_matrix[mapping[i]])
    edges_threshold[mapping[i]] = np.median(hub) + np.std(hub)

print(edges_threshold)


{'theta': 5.7887889396653875, 'alpha': 4.946793808531571, 'beta': 4.629307785239494}


In [112]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(n_channels):
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > edges_threshold[mapping[i]]:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

{'theta': ['C4', 'AF3', 'F7', 'P7'], 'alpha': ['C4', 'Fp1', 'AF3', 'F7', 'CP5', 'P7'], 'beta': ['F4', 'AF3', 'CP5', 'P7']}


Create dictionary of Regions in brain vs channels

In [113]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [114]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

print(selected_regions)

{'theta': ['LPO', 'LF', 'RC'], 'alpha': ['LPO', 'LF', 'LC', 'RC'], 'beta': ['LPO', 'LF', 'RF', 'LC']}


### 7. Network Metrics for selected regions for selected bands


Boxplot helps in reading median, interquartile range and outliers
Box : middle 50% of all data lies (Interquartile range)
Lower end: 1st quartile
Upper end: 3rd quartile
Solid line: Median
Dashed line: Mean
T shaped whiskers: Last point 1.5 times the interquartile range (max and min without outliers)
Points: Further out are outliers

In [91]:
# for i in mapping:
#     #group 1 measures
#     g1 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP1}-{mapping[i]}']))
#     hub = calculate_hub(binarized_matrix[f'{GROUP1}-{mapping[i]}'])
#     g1_group_measures = pd.DataFrame({
#         'swn': g1[3]/g1[0],
#         'hub': hub,
#         'asymmetry': a1[mapping[i]],
#         'links': l1[mapping[i]]
#     })

#     # group 2 measures
#     g2 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}']))
#     hub = calculate_hub(binarized_matrix[f'{GROUP2}-{mapping[i]}'])
#     g2_group_measures = pd.DataFrame({
#         'swn': g2[3]/g2[0],
#         'hub': hub,
#         'asymmetry': a2[mapping[i]],
#         'links': l2[mapping[i]]
#     })

#     fig, axes = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
#     parameters = ['swn', 'hub', 'asymmetry', 'links']

#     for i, param in enumerate(parameters):
#         print("&&&&&", g1_group_measures[param], g2_group_measures[param], GROUP1, GROUP2)
#         ax = axes[i]
#         ax.boxplot([g1_group_measures[param], g2_group_measures[param]], labels=[GROUP1, GROUP2])
#         ax.set_ylabel(param)

#     plt.tight_layout()
#     plt.show()

#     # Perform t-test for each parameter
#     for param in parameters:
#         t_stat, p_value = ttest_rel(g1_group_measures[param], g2_group_measures[param])
#         print(f"Parameter: {param}, p-value: {p_value}")